# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [ ]:
DATA_FOLDER = 'data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.
EBOLA = DATA_FOLDER + "/ebola"
GUINEA = EBOLA + "/guinea_data"
LIBERIA = EBOLA + "/liberia_data"
SIERRA_LEONE = EBOLA + "/sl_data"

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average* per year of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [ ]:
sierra_descriptions = set([b for a,b,c, in list(preprocessed_sierra.index.values)])
print("Sierra: {}\n".format(sierra_descriptions))

guinea_descriptions = set([b for a,b,c, in list(preprocessed_guinea.index.values)])
print("Guinea: {}\n".format(guinea_descriptions))

liberia_descriptions = set([b for a,b,c, in list(preprocessed_liberia.index.values)])
print("Liberia: {}\n".format(liberia_descriptions))
#sierra[~sierra['34 Military Hospital'].isnull()]
#preprocessed_sierra.iloc[0]


### 1.1 Importing the data

First of all, we will split the cleaning in three parts, one for each country, since the files were consistent for each country. Hence, we start by loading all of the data from each folder into three different DataFrames. 

In [ ]:
from IPython.core.display import display, HTML
import pandas as pd
import numpy as np
import glob

# A few helper functions

"""
Returns a pandas dataframe from a folder full of 
csv files.
"""
def create_folder_data_frame(FOLDER):
    list_ = []
    for file_ in glob.glob(FOLDER + "/*.csv"):
        df = pd.read_csv(file_, index_col=None)
        list_.append(df)
    return pd.concat(list_)

"""
Sets a new column "Country" that becomes part of a multi index
afterwards, and returns the indexed frame.
"""
def index_and_country(data_frame, country):
    data_frame["Country"] = country
    data_frame.set_index(['Date', 'Description', 'Country'], inplace=True)
    return data_frame
    
"""
We preprocess each Country one after the other and set their indices
to a MultiIndex of Date, Description and Country, with each column
mapped to one of these indices.
"""
def preprocess(FOLDER, name, columns=None):
    frame = create_folder_data_frame(FOLDER)
    if columns:
        frame.rename(columns=columns, inplace=True)
    frame.Date = pd.to_datetime(frame.Date)
    frame = index_and_country(frame, name)
    return frame
    
preprocessed_guinea = preprocess(GUINEA, "Guinea")

liberia_column_mapping = {"Variable": "Description", "National": "Totals", 'Unnamed: 18': "Unknown"}
preprocessed_liberia = preprocess(LIBERIA, "Liberia", liberia_column_mapping)

sierra_column_mapping = {"variable": "Description", "National": "Totals", 'Unnamed: 18': "Unknown", "date": "Date"}
preprocessed_sierra = preprocess(SIERRA_LEONE, "Sierra Leone", sierra_column_mapping)

### 1.2 Cleaning up

Now that we have our three DataFrames, we need to merge them into a single one in order to query it to get the daily average deaths and cases. What we need to do in order to merge them, is to clean each DataFrame, and create a consistency inbetween columns after choosing which columns to use.

After taking a look at the data, we decided to deliver the daily average of new cases and new deaths as a sum of the following categories: the suspected, probables, and confirmed cases and deaths, hence we only kept for each DataFrame the right columns. When the "right" columns are not perfectly defined, we explain which one we chose and why. The following set of columns are preselected after looking at the data, the other columns were either inconsistent or made no sense whatsoever.

#### 1.2.1 Guinea

In [ ]:
# The columns that are of interest to us, mapped
# to their centralized names for the final DataFrame
interest = { 'New cases of suspects': 'new_suspected',
        'New cases of probables': 'new_probable',
        'New cases of confirmed': 'new_confirmed',
        'Total deaths (confirmed + probables + suspects)': 'deaths',
        'Total deaths of suspects': 'death_suspected',
        'Total deaths of probables': 'death_probable',
        'Total deaths of confirmed': 'death_confirmed' }

    
# We start by keeping only the interesting columns for the task
interest_indices = preprocessed_guinea.index.get_level_values('Description').isin(interest.keys())
guinea_df = preprocessed_guinea[interest_indices]

Now here is the first assumption we made regarding the data of Guinea, there are no missing values in the totals of deaths, but there are some NaN values in the New cases. We figured that of these new cases, NaN meant that there were either no data for the day, or that the data was not entered. It seems more plausible that people did not fill the data when there was no new cases, so we went with that explanation and changed all NaN values to 0. Also, since the number of cases and deaths can only be discrete we converted the dataframe to integers.

In [ ]:
guinea_df = guinea_df.fillna(0).astype(int)
guinea_df = guinea_df.sort_index(0)

By verifying that the sum of cities and the total column are the same, we can see that there are some discrepencies. We then decided to remove the rows where the discrepencies are too high, namely 10% compared to the Totals column. Otherwise we keep the row, and we finally decided to keep mean of both columns, as they were quite similar, and it was impossible to split them apart and choose one. So the mean is the most obvious merging tool. 

In [ ]:
# Keeping only the interesting columns, namely one for the Total
# and one for the total of cities, to filter out unreliable data points
guinea_df['Cities_total'] = guinea_df.sum(1) - guinea_df.Totals
guinea_df = guinea_df[['Cities_total', 'Totals']]
equality = sum(guinea_df['Cities_total'] == guinea_df['Totals']) == 0
print("Does the sum of cities equal the Totals columm ? {}".format(equality))

# Now we want to remove the rows for which the total of cities 
# diverges from the Totals column of at least 10%
guinea_df = guinea_df[np.abs(guinea_df.Cities_total - guinea_df.Totals) <= (0.1 * guinea_df.Totals)]
guinea_df['Totals'] = guinea_df.mean(1)
guinea_df = guinea_df.Totals

# Converting the indices to columns, and readjusting the indices
guinea_df = guinea_df.unstack('Description', fill_value=0)
guinea_df.rename(columns=interest, inplace=True)

display(HTML(guinea_df.to_html()))

Looking at the total deaths, we can see that there is a row where we have missing data, we assumed here that the person forgot to write the measurements, hence we decided to fill the row with the mean of the previous and next measurements. That would simulate a linear increase and seems reasonable for one data point.

In [ ]:
# Filling the row with the mean of the previous and next measurements
# We also remove the cumulative sum, in order to get the daily data
for c in guinea_df.columns:
    if "death_" in c:
        guinea_df[c].loc[('2014-09-26', 'Guinea')] = int((guinea_df[c].loc[('2014-09-24', 'Guinea')] + guinea_df[c].loc[('2014-09-30', 'Guinea')]) / 2)
        guinea_df[c] = guinea_df[c] - guinea_df[c][0]
        guinea_df[c][1:] = guinea_df[c][1:].copy().as_matrix() - guinea_df[c][:-1].copy().as_matrix()
        
# And finally, since the data can get negative because of the previous
# computations we made, we can set them back to 0
guinea_df[guinea_df < 0] = 0

Finally we can take a look at the sum of all 3 categories into one. But first, we need to choose how to process the fact that we can pick either from the deaths column or the sum of the three other categories of deaths. Let's take a look at the difference between the two.

In [ ]:
guinea_df['new_cases'] = guinea_df[['new_suspected', 'new_confirmed', 'new_probable']].sum(axis=1)
guinea_df['deaths_summed'] = guinea_df[['death_confirmed', 'death_probable', 'death_suspected']].sum(axis=1)
# We reverse the cumulative sum on the deaths column to get a daily value
guinea_df['deaths'] = guinea_df['deaths'] - guinea_df['deaths'][0]
guinea_df['deaths'][1:] = guinea_df['deaths'][1:].copy().as_matrix() - guinea_df['deaths'][:-1].copy().as_matrix()

display(HTML(guinea_df[['deaths', 'deaths_summed']].to_html()))

Taking a look at the column deaths and the column death_summed, we can clearly see that they are about the same, besides a few discrepencies and two or three clearly wrong values. Hence, we decided to only retain the deaths_summed column that seemed more consistent.

In [ ]:
# We now have a clean DataFrame to work with:
guinea_df = guinea_df[['new_cases', 'deaths_summed']]
guinea_df.rename(columns={'deaths_summed':'deaths'}, inplace=True)

display(HTML(guinea_df.to_html()))

#### 1.2.2 Sierra Leone

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here